In [4]:
import numpy as np
import sympy as sym

In [5]:
A = np.array([[1,0,0],[5,1,0],[-2,3,1]])
B = np.array([[4,-2,1],[0,3,7],[0,0,2]])


#3,4,7,9
def GetMultiplication(A,B):
    matrix_column = []
    for i in range(len(B)):
        column = []
        for j in range(len(B)): 
            column.append(B[j][i])
        matrix_column.append(column)
    

    multiplication_matrix =[]
    for k in A:
        new_matrix = []
        for w in matrix_column:
            new_matrix.append(np.dot(k,w))
        multiplication_matrix.append(new_matrix)
        
    return np.array(multiplication_matrix)

GetMultiplication(A,B)

array([[ 4, -2,  1],
       [20, -7, 12],
       [-8, 13, 21]])

In [9]:
#7
M = np.array([[4,-2,1],[20,-7,12],[-8,13,17]])

def FactorizationLU(m):
    A = np.array(m)
    dim = A.shape
    L = np.zeros(dim)
    U = np.zeros(dim)
    n = dim[0]
    
    for j in range(n):
        L[j][j] = 1.0
        
        for i in range(j+1,n):
            L[i][j] = A[i][j] / A[j][j]
            
            for k in range(j+1,n):
                A[i][k] = A[i][k] - L[i][j]*A[j][k]
                
        for k in range(j,n):
            U[j][k] = A[j][k]
    
    return L,U

L,U = FactorizationLU(M)
print(L)
print(U)

comprobacion = np.dot(L,U)
print(comprobacion)

[[ 1.  0.  0.]
 [ 5.  1.  0.]
 [-2.  3.  1.]]
[[ 4. -2.  1.]
 [ 0.  3.  7.]
 [ 0.  0. -2.]]
[[ 4. -2.  1.]
 [20. -7. 12.]
 [-8. 13. 17.]]


In [138]:
C = np.array([[4,1,1],[1,3,2],[1,2,5]])
np.linalg.eig(C)

(array([6.89510652, 3.39729507, 1.70759841]),
 array([[ 0.43170413,  0.88573564,  0.17059871],
        [ 0.49725362, -0.07589338, -0.86427949],
        [ 0.75257583, -0.45794385,  0.47319874]]))

In [150]:
#9 usando funciones raras de numpy y arctan2

def jacobi_eig(A, tol=1e-10, maxiter=1000):
    n = A.shape[0]
    D = A.copy()
    Q = np.eye(n)

    off_diag = np.abs(np.triu(D, 1)).max()
    i = 0
    while off_diag >= tol and i < maxiter:
        # Encuentra los índices del elemento no diagonal máximo
        idx_max = (0, 0)
        max_val = -np.inf
        for row in range(n):
            for col in range(row+1, n):
                if np.abs(D[row, col]) > max_val:
                    max_val = np.abs(D[row, col])
                    idx_max = (row, col)
        row, col = idx_max

        # Calcula la matriz de rotación
        #arctan2ayuda al signo del cuadrante
        theta = 0.5 * np.arctan2(2 * D[row, col], D[col, col] - D[row, row])
        c = np.cos(theta)
        s = np.sin(theta)
        R = np.eye(n)
        R[row, row] = c
        R[col, col] = c
        R[row, col] = s
        R[col, row] = -s

        # Actualiza las matrices Q y D
        D = np.dot(R.T, np.dot(D, R))
        Q = np.dot(Q, R)

        off_diag = np.abs(np.triu(D, 1)).max()
        i += 1

    eigvals = np.flip(np.diag(D))
    eigvecs = Q[:,::-1]

    return eigvals, eigvecs

jacobi_eig(C)

(array([6.89510652, 3.39729507, 1.70759841]),
 array([[ 0.43170413,  0.88573564,  0.17059871],
        [ 0.49725362, -0.07589338, -0.86427949],
        [ 0.75257583, -0.45794385,  0.47319874]]))